In [3]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
import pickle
import itertools
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon as MplPolygon
import chronnet_utils as cu 
import warnings
from scipy.linalg import eigvals
from scipy import optimize
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression
from datetime import datetime
import networkx as nx
from collections import Counter
from sis_steady_state_and_eval import jsd_from_samples,nimfa_sis_steady_state_root_1,recognition_quality

from scipy.stats import spearmanr, kendalltau
from sklearn.metrics import ndcg_score, average_precision_score




from compute_y_true import compute_y_true_metrics as compute_y_true
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")




In [4]:
with open('chronnet_graph_list_20250615_141938.pkl', 'rb') as f:
    chronnet_graph_list = pickle.load(f)

with open('burning_result_list.pkl', 'rb') as f:
    burning_result_list = pickle.load(f)

all_y_true_sis = {
    item['grid_size']: item['y_true_sis']
    for item in burning_result_list
}
all_y_true_sf = {
    item['grid_size']: item['y_true_sf']
    for item in burning_result_list     
}
all_y_true_net = {
    item['grid_size']: item['y_true_network']       
    for item in burning_result_list
}
all_y_true = {
    item['grid_size']: item['y_true']
    for item in burning_result_list 
}

# Tau Experiment

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

grid_sizes = [2000, 3000, 5000]


network_list = {
    entry['grid_size']: entry['chronnet_pruned']
    for entry in chronnet_graph_list
}

gamma = 1.0
tau_c = 1.0
tau_vals = np.linspace(1.1 * tau_c, 100 * tau_c, num=100)


sis_results_list = []

for grid_size in grid_sizes:
    G = network_list.get(grid_size)

    sccs = list(nx.strongly_connected_components(G))
    top2_sccs = sorted(sccs, key=len, reverse=True)[:2]

    
    for rank, scc_nodes in enumerate(top2_sccs, start=1):
        subG = G.subgraph(scc_nodes).copy()
        # remove self-loops
        subG.remove_edges_from(list(nx.selfloop_edges(subG)))
        
        cc_node_list = list(subG.nodes())
        W = nx.to_numpy_array(subG, nodelist=cc_node_list, weight='weight', dtype=float)
        lambda_max = np.max(np.abs(np.linalg.eigvals(W)))
        W_norm = W / lambda_max

        for tau in tau_vals:
            beta = tau * gamma
            print(f"[grid {grid_size} | SCC {rank}] tau={tau:.3f}, beta={beta:.3f}")

            v_sis = nimfa_sis_steady_state_root_1(W_norm, beta, gamma)
            if np.all(v_sis == 0):
                print(f"[grid {grid_size} | SCC {rank}] tau={tau:.3f} not converged, skip")
                continue
            


            y_true = np.array([all_y_true[grid_size].get(node, 0) for node in cc_node_list], dtype=float)
            y_true_sf = np.array([all_y_true_sf[grid_size].get(node, 0) for node in cc_node_list], dtype=float)
            y_true_sis = np.array([all_y_true_sis[grid_size].get(node, 0) for node in cc_node_list], dtype=float)
            y_true_network = np.array([all_y_true_net[grid_size].get(node, 0) for node in cc_node_list], dtype=float)
            v_hybrid = y_true_sf + (1 - y_true_sf) * v_sis
            
            
            # hybrid
            rq_val         = recognition_quality(v_hybrid, y_true)
            jsd_val        = jsd_from_samples(v_hybrid, y_true, scale='minmax')
            mse            = np.mean((v_hybrid - y_true) ** 2)
            rmse           = np.sqrt(mse)

            # net # jsd_val_net    = jsd_from_samples(v_sis, y_true_sis, scale='none')
            rq_val_net     = recognition_quality(v_sis, y_true_sis)
            mse_net        = np.mean((v_sis - y_true_sis) ** 2)
            rmse_net       = np.sqrt(mse_net)
            jsd_val_net_nm = jsd_from_samples(v_sis, y_true_sis, scale='minmax')
            
            # Compute Kendall and Spearman correlations
            
            rho=spearmanr(v_sis, y_true_sis).statistic
            p_value=spearmanr(v_sis, y_true_sis).pvalue
    
            nd_50= ndcg_score([y_true_sis], [v_sis], k=50)
            nd_100 = ndcg_score([y_true_sis], [v_sis], k=100)
            nd_300 = ndcg_score([y_true_sis], [v_sis], k=300)
            nd_500 = ndcg_score([y_true_sis], [v_sis], k=500)
            nd_1000 = ndcg_score([y_true_sis], [v_sis], k=1000)


            # compute the topk% ndcg
            k1= int(len(v_sis) * 0.01)
            k5= int(len(v_sis) * 0.05)
            k10= int(len(v_sis) * 0.1)
            k20= int(len(v_sis) * 0.2)
            k50= int(len(v_sis) * 0.5)

            nd_1p = ndcg_score([y_true_sis], [v_sis], k=k1)
            nd_5p = ndcg_score([y_true_sis], [v_sis], k=k5)
            nd_10p = ndcg_score([y_true_sis], [v_sis], k=k10)
            nd_20p = ndcg_score([y_true_sis], [v_sis], k=k20)
            nd_50p = ndcg_score([y_true_sis], [v_sis], k=k50)

            
            sis_results_list.append({
                'grid_size': grid_size,
                'scc_rank':      rank,
                'tau':           tau,
                'beta':          beta,
                'gamma':         gamma,
                'cc_node_list': cc_node_list,

                'RQ':            rq_val,
                'JSD':           jsd_val,
                'MSE':           mse,
                'RMSE':          rmse,
                'RQ_net':        rq_val_net,
                'MSE_net':       mse_net,
                'RMSE_net':      rmse_net,
                'JSD_net_norm':  jsd_val_net_nm,

                'rho':           rho,
                'p_value':       p_value,
                'ndcg_50':       nd_50,
                'ndcg_100':      nd_100,
                'ndcg_300':      nd_300,
                'ndcg_500':      nd_500,
                'ndcg_1000':     nd_1000,
                'ndcg_1p':       nd_1p,
                'ndcg_5p':       nd_5p,
                'ndcg_10p':      nd_10p,
                'ndcg_20p':      nd_20p,
                'ndcg_50p':      nd_50p,

                'v_sis':         v_sis,
                'v_hybrid':      v_hybrid,

                'y_true':     y_true,
                'y_true_sis': y_true_sis,
                'y_true_sf':  y_true_sf,
                'y_true_net': y_true_network
               
            })

 


In [6]:
# save to a pickle file 
with open(f'sis_results_list_wo_selfloop.pkl', 'wb') as f:
    pickle.dump(sis_results_list, f)

In [ ]:
filename = 'chronnet_graph_list.pkl'
with open(filename, 'rb') as f:
    chronnet_graph_list = pickle.load(f)
    
grid_sizes = [2000, 3000, 5000]

network_list = {
    entry['grid_size']: entry['chronnet_pruned']
    for entry in chronnet_graph_list
}

# node_metrics_list 
with open('node_metrics_list.pkl', 'rb') as f:
    node_metrics_list = pickle.load(f)

# chronnet_result_list
with open('chronnet_result_list.pkl', 'rb') as f:
    chronnet_result_list = pickle.load(f) 

In [9]:
# compute eigenvector centrality for the top 2 SCCs in each grid size

scc_node_result = []
grid_sizes = [2000, 3000, 5000]

for grid_size in grid_sizes:

    G = network_list.get(grid_size)
    sccs = list(nx.strongly_connected_components(G))
    top2_sccs = sorted(sccs, key=len, reverse=True)[:2]

    y_true_sis = all_y_true_sis.get(grid_size, {})
    y_true_sf = all_y_true_sf.get(grid_size, {})
    y_true_network = all_y_true_net.get(grid_size, {})      
    y_true = all_y_true.get(grid_size, {})  

    size_to_gdf = {
    entry['grid_size']: entry['gdf_sjoined']
    for entry in chronnet_result_list
    }
    
    gdf_sjoined = size_to_gdf.get(grid_size, pd.DataFrame())

    for rank, scc_nodes in enumerate(top2_sccs, start=1):
        subG = G.subgraph(scc_nodes).copy()
        # remove self-loops
        subG.remove_edges_from(list(nx.selfloop_edges(subG)))
        
        cc_node_list = list(subG.nodes())

        # Compute node metrics
        in_deg_dict = dict(subG.in_degree())
        out_deg_dict = dict(subG.out_degree())
        in_str_dict = dict(subG.in_degree(weight='weight'))
        out_str_dict = dict(subG.out_degree(weight='weight'))
        deg_dict = dict(subG.degree())
        str_dict = dict(subG.degree(weight='weight'))

        ### !!!!! scc
        scc_y_true_list= compute_y_true(
            gdf_sjoined,
            subG,
            cell_col='cell',
            time_col='time_group',
            lag_hours=12
        )
        scc_y_true_dict=scc_y_true_list['y_true_dict']
        scc_y_true_sf_dict=scc_y_true_list['y_true_sf_dict']
        scc_y_true_network_dict=scc_y_true_list['y_true_network_dict']
        scc_y_true_sis_dict=scc_y_true_list['y_true_sis_dict']

  


        subG_undir = subG.to_undirected() # ! only one edge is created with an arbitrary choice of which edge data to use.
        ev_dict = nx.eigenvector_centrality(
            subG,
            max_iter=1000,
            tol=1e-6
            #,
            #weight='weight'
        )
        ev_dict_w=nx.eigenvector_centrality(
            subG,
            max_iter=1000,
            tol=1e-6,
            weight='weight'
        )
        betweeness_dict = nx.betweenness_centrality(
            subG,
            normalized=True
          #  weight='weight'
        )
        betweeness_dict_w = nx.betweenness_centrality(
            subG,
            normalized=True,
            weight='weight'
        )
        closeness_dict = nx.closeness_centrality(
            subG
          #  distance='weight'
        )
        closeness_dict_w = nx.closeness_centrality(
            subG,
            distance='weight'
        )
        clustering_coeffs = nx.clustering(subG_undir)
        clastering_coeffs_w = nx.clustering(subG_undir, weight='weight')   

        # NetworkX's PageRank already uses a row‑stochastic transition matrix (out‑degree normalised).
        pagerank_dict = nx.pagerank(subG, alpha=0.85, max_iter=100, tol=1e-6,weight=None)
        pagerank_dict_w = nx.pagerank(subG, alpha=0.85, max_iter=100, tol=1e-6, weight='weight') 

        scc_node_metrics = pd.DataFrame({
            'cell': cc_node_list,
            'degree': [deg_dict[cell] for cell in cc_node_list],
            'strength': [str_dict[cell] for cell in cc_node_list],
            'in_degree': [in_deg_dict[cell] for cell in cc_node_list],
            'out_degree': [out_deg_dict[cell] for cell in cc_node_list],
            'in_strength': [in_str_dict[cell] for cell in cc_node_list],
            'out_strength': [out_str_dict[cell] for cell in cc_node_list],
            'clustering_coefficient': [clustering_coeffs[cell] for cell in cc_node_list],
            'eigenvector_centrality': [ev_dict[cell] for cell in cc_node_list],
            'betweenness_centrality': [betweeness_dict[cell] for cell in cc_node_list],
            'closeness_centrality': [closeness_dict[cell] for cell in cc_node_list],
            'eigenvector_centrality_w': [ev_dict_w[cell] for cell in cc_node_list],
            'betweenness_centrality_w': [betweeness_dict_w[cell] for cell in cc_node_list],
            'closeness_centrality_w': [closeness_dict_w[cell] for cell in cc_node_list],
            'clustering_coefficient_w': [clastering_coeffs_w.get(cell, 0) for cell in cc_node_list],
            'pagerank': [pagerank_dict[c] for c in cc_node_list],
            'pagerank_w': [pagerank_dict_w[c] for c in cc_node_list],
            
            'v_true_sis': [y_true_sis.get(cell, 0) for cell in cc_node_list],
            'v_true_net': [y_true_network.get(cell, 0) for cell in cc_node_list],
            'v_true_sf': [y_true_sf.get(cell, 0) for cell in cc_node_list],
            'v_true': [y_true.get(cell, 0) for cell in cc_node_list],

            'v_true_scc': [scc_y_true_dict.get(cell, 0) for cell in cc_node_list],
            'v_true_scc_net': [scc_y_true_network_dict.get(cell, 0) for cell in cc_node_list],
            'v_true_scc_sf': [scc_y_true_sf_dict.get(cell, 0) for cell in cc_node_list],    
            'v_true_scc_sis': [scc_y_true_sis_dict.get(cell, 0) for cell in cc_node_list]

        })
        
        scc_node_result.append({
            'grid_size': grid_size,
            'scc_rank': rank,
            'node_metrics': scc_node_metrics
        })






In [ ]:
# compute JSD , recognition quality between node metrics and v_true_sis 

scc_eval_list = []


for rec in scc_node_result:
    node_metrics = rec['node_metrics']
    v_true_sis = node_metrics['v_true_sis'].values
    for metric in ['degree','strength','eigenvector_centrality','eigenvector_centrality_w', 'betweenness_centrality', 'closeness_centrality','pagerank', 'pagerank_w']:
        v_metric = node_metrics[metric].values
        jsd_val = jsd_from_samples(v_metric, v_true_sis, scale='minmax')
        rq_val = recognition_quality(v_metric, v_true_sis)


        rho = spearmanr(v_metric, v_true_sis).correlation
        ken = kendalltau(v_metric, v_true_sis).correlation
        ndcg_100 = ndcg_score([v_metric], [v_true_sis], k=100)
        ndcg_300 = ndcg_score([v_metric], [v_true_sis], k=300)
        ndcg_500 = ndcg_score([v_metric], [v_true_sis], k=500)
        ndcg_1000= ndcg_score([v_metric], [v_true_sis], k=1000)
        k1 = int(len(v_metric) * 0.01)
        k5 = int(len(v_metric) * 0.05)
        k10 = int(len(v_metric) * 0.1)
        k20 = int(len(v_metric) * 0.2)
        k50 = int(len(v_metric) * 0.5)  
        nd_1p = ndcg_score([v_metric], [v_true_sis], k=k1)
        nd_5p = ndcg_score([v_metric], [v_true_sis], k=k5)
        nd_10p = ndcg_score([v_metric], [v_true_sis], k=k10)
        nd_20p = ndcg_score([v_metric], [v_true_sis], k=k20)        
        nd_50p = ndcg_score([v_metric], [v_true_sis], k=k50)


        scc_eval_list.append({
            'grid_size': rec['grid_size'],
            'scc_rank': rec['scc_rank'],
            'metric': metric,
            'JSD': jsd_val,
            'RQ': rq_val,
            'rho': rho,
            'ken': ken,
            'ndcg_100': ndcg_100,
            'ndcg_300': ndcg_300,
            'ndcg_500': ndcg_500,
            'ndcg_1000': ndcg_1000,
            'ndcg_1p': nd_1p,
            'ndcg_5p': nd_5p,
            'ndcg_10p': nd_10p,
            'ndcg_20p': nd_20p,
            'ndcg_50p': nd_50p
        })



scc_eval_df = pd.DataFrame(scc_eval_list)
scc_eval_df

# Performance with the best tau

In [ ]:
import pandas as pd
import numpy as np



RQ_COL  = "RQ_net"
JSD_COL = "JSD_net_norm"
TAU_COL = "tau"

def pick_tau_turning_point(df,
                           rq_plateau_thresh=2e-3,   # |ΔRQ| below this ⇒ flat
                           plateau_window=2,         # how many flat steps in a row
                           rq_plateau_frac=0.95):    # fallback: ≥95 % of max RQ
    """
    Given a (grid_size, scc_rank) sub‑DataFrame, return one row containing
    tau_opt, RQ_net, JSD_net_norm that best trades off high‑and‑flat RQ vs
    still‑low JSD.
    """
    # 1. average repeated runs at the same τ
    agg = (df.groupby(TAU_COL)
             .agg({RQ_COL: "mean", JSD_COL: "mean"})
             .sort_index()
             .reset_index())

    # If we only have a couple of τ values, just grab the “best” one.
    if len(agg) < 3:
        return agg.sort_values([RQ_COL, JSD_COL], ascending=[False, True]).iloc[0]

    # 2. finite differences
    agg["dJSD"] = agg[JSD_COL].diff()
    agg["dRQ"]  = agg[RQ_COL].diff()

    best = None
    # 3. turning point: JSD just turned upward and RQ has been flat
    for i in range(plateau_window, len(agg)):
        jsd_turn = agg.loc[i - 1, "dJSD"] <= 0 and agg.loc[i, "dJSD"] > 0
        rq_flat  = (agg.loc[i - plateau_window + 1 : i, "dRQ"]
                      .abs()
                      .lt(rq_plateau_thresh)
                      .all())
        if jsd_turn and rq_flat:
            best = agg.loc[i, [TAU_COL, RQ_COL, JSD_COL]]
            break
            
        if best is None:
            max_rq = agg[RQ_COL].max()
            plateau = agg[agg[RQ_COL] >= rq_plateau_frac * max_rq]
            best = plateau.nsmallest(1, JSD_COL).iloc[0]

    better = agg[
        (agg[JSD_COL] < best[JSD_COL]) &
        (agg[RQ_COL] > best[RQ_COL])
    ]
    if not better.empty:
        best = better.sort_values(RQ_COL, ascending=False).iloc[0]

    return best[[TAU_COL, RQ_COL, JSD_COL]]



# ------------------------------------------------------------------ main driver
# convert the list to a DataFrame and then to csv
import pandas as pd
import pickle
with open('sis_results_list_wo_selfloop.pkl', 'rb') as f:
    sis_results_list = pickle.load(f)
df = pd.DataFrame(sis_results_list)


best_rows = []
for (gsize, rank), sub in df.groupby(["grid_size", "scc_rank"]):
    best = pick_tau_turning_point(sub)
    best_rows.append({
        "grid_size":      gsize,
        "scc_rank":       rank,
        "tau_opt":        best[TAU_COL],
        RQ_COL:           best[RQ_COL],
        JSD_COL:          best[JSD_COL]
    })

results = pd.DataFrame(best_rows).sort_values(["grid_size", "scc_rank"])
print(results.to_string(index=False))

# If you need it on disk:
results.to_csv("best_tau_by_grid_scc_wo_selfloop.csv", index=False)


In [ ]:
import pandas as pd
import pickle



with open('sis_results_list_wo_selfloop.pkl', 'rb') as f:
    sis_results_list = pickle.load(f)   


scc_df = pd.DataFrame(sis_results_list) #scc_df = pd.read_csv('scc_results.csv')

best_tau_df = pd.read_csv('best_tau_by_grid_scc_wo_selfloop.csv')
merged_rows = []

for _, row in best_tau_df.iterrows():
    g, r, tau_opt = row['grid_size'], row['scc_rank'], row['tau_opt']

    
    match = scc_df[
        (scc_df['grid_size'] == g) &
        (scc_df['scc_rank'] == r) &
        (scc_df['tau'].round(6) == round(tau_opt, 6))  
    ]

    if match.empty:
        print(f"Warning: no match found for grid={g}, rank={r}, tau={tau_opt}")
        continue

   
    merged = match.mean(numeric_only=True).to_dict()
    merged['grid_size'] = g
    merged['scc_rank'] = r
    merged['tau_opt'] = tau_opt

    merged_rows.append(merged)


final_df = pd.DataFrame(merged_rows)
final_df = final_df.sort_values(['grid_size', 'scc_rank'])

final_df



In [ ]:

colmns=['grid_size', 'scc_rank', 'tau_opt', 'RQ_net', 'JSD_net_norm',
         'ndcg_100', 'ndcg_300','ndcg_500', 'ndcg_1000']
final_df1 = final_df[colmns]
final_df1.to_csv('ablation_study_wo_selfloop.csv', index=False)

